# Grove Analog ports and PWM

The arduino shield has dedicated analog ports which can be accessed through the `Analog` peripheral. This peripheral can only be used with the A1 to A4 ports on the PYNQ shield and not at all using the PMOD interface. First we'll load the library, initialise the base overlay and print the API for the `Analog` peripheral.

In [1]:
import ipython_microblaze as ipmb

from pynq.overlays.base import BaseOverlay
base = BaseOverlay('base.bit')

print(ipmb.Analog.declaration)


typedef int analog;

analog analog_connect(unsigned char port, unsigned char wire);
int analog_read_raw(analog port);
float analog_read(analog port);




This peripheral follows the format of many in that upon connecting the pin, a handle is returned which can be used for reading the values back. `analog_read_raw` returns a number between 0 and 65536 where 65536 is equivalent to 3.3 Volts. `analog_read` does the conversion to Volts internally.

For a simple example, let's create a program which reads the value from the A1 port and sends it back to python.

In [3]:
%%microblaze base.ARDUINO analog_test ipmb.Analog
#include <unistd.h>

int main() {
    analog conn = analog_connect(A1, 0);
    int val = analog_read_raw(conn);
    write(STDOUT_FILENO, &val, 4);
}

In [8]:
import struct
struct.unpack('I', analog_test.stream.read())

analog_test.reset()

Next we can introduce the PWM Peripheral which generates a regular stream of pulses on an output pin.

In [5]:
print(ipmb.PWM.declaration)


typedef int pwm;

pwm pwm_connect(unsigned char port, unsigned char wire);
void pwm_generate_cycles(pwm timer, int period, int pulse);
void pwm_generate_us(pwm timer, int period, int pulse);
void pwm_stop(pwm timer);



This peripheral driver uses the same idiom as the `Analog` driver. The PWM has some limitations to be aware of - for the Arduino output it can only be connected to digital pins D3, D5, D6, D9, D10 and D11 however all of these outputs can be used in parallel. For the PYNQ shield this restricts single-wire Grove peripherals to the G2, G4 and G6 connectors.

The `pwm_generate_cycles` and `pwm_generate_us` functions start the PWM generator and set the period and pulse width - the pulse width is given in the same units as the period. `pwm_stop` drives the output at 0.

For an example we can connect a sensor to A1 and an output such as an LED to G2 and use that to represent the current value of the input.

In [9]:
%%microblaze base.ARDUINO pwm_test ipmb.PWM ipmb.Analog

int main() {
    analog input = analog_connect(A1, 0);
    pwm output = pwm_connect(G2, 0);
    int val = analog_read(input);
    while (1) {
        val = analog_read_raw(input);
        pwm_generate_cycles(output, 65536, val);
    }
}

In [10]:
pwm_test.reset()

The PWM library can also be used on a PMOD output although here there is only a single generator but it can be routed to any of the output pins. This example ramps an LED attached to G2

In [11]:
%%microblaze base.PMODA pwm_test ipmb.PWM

#include <pmod.h>
#include <unistd.h>

int main() {
    pwm output = pwm_connect(G2, 0);
    while (1) {
        for (int i = 0; i < 100; ++i) {
            pwm_generate_cycles(output, 100, i);
            for (int j = 0; j < 1000000; ++j);
            
        }
    }
}

In [12]:
pwm_test.reset()